In [1]:
pip install sentence-transformers scikit-learn pandas numpy

   ---------------------------------------- 0.0/204.2 MB ? eta -:--:--
   ---------------------------------------- 1.3/204.2 MB 6.1 MB/s eta 0:00:34
    --------------------------------------- 3.1/204.2 MB 8.0 MB/s eta 0:00:26
    --------------------------------------- 4.5/204.2 MB 7.3 MB/s eta 0:00:28
   - -------------------------------------- 5.2/204.2 MB 6.5 MB/s eta 0:00:31
   - -------------------------------------- 6.6/204.2 MB 6.3 MB/s eta 0:00:32
   - -------------------------------------- 7.6/204.2 MB 6.1 MB/s eta 0:00:33
   - -------------------------------------- 8.9/204.2 MB 6.1 MB/s eta 0:00:33
   - -------------------------------------- 10.0/204.2 MB 6.0 MB/s eta 0:00:33
   -- ------------------------------------- 11.0/204.2 MB 5.9 MB/s eta 0:00:33
   -- ------------------------------------- 11.8/204.2 MB 5.6 MB/s eta 0:00:35
   -- ------------------------------------- 12.6/204.2 MB 5.4 MB/s eta 0:00:36
   -- ------------------------------------- 13.4/204.2 MB 5.4 MB/s 

In [6]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


In [7]:
# Load pre-trained embedding model (FREE)
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# Load datasets (each file contains one article per line)
with open("Fake.csv", "r", encoding="utf-8") as f:
    fake_news = f.readlines()

with open("True.csv", "r", encoding="utf-8") as f:
    real_news = f.readlines()


In [8]:
# Create DataFrame
df_fake = pd.DataFrame({"text": fake_news, "label": 1})  # Fake news = 1
df_real = pd.DataFrame({"text": real_news, "label": 0})  # Real news = 0

df = pd.concat([df_fake, df_real], ignore_index=True)

# Generate embeddings
df["embeddings"] = df["text"].apply(lambda x: model.encode(x.strip()))

In [9]:
# Prepare training data
X = np.vstack(df["embeddings"].values)
y = df["label"].values

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
# Train classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [11]:
# Predict on test set
y_pred = clf.predict(X_test)

# Evaluate model
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.2f}")


Model Accuracy: 0.93


In [26]:
# Example Prediction
def predict_fake_news(text):
    embedding = model.encode(text)
    prediction = clf.predict([embedding])[0]
    return "Fake News" if prediction == 1 else "Real News"

# Test exampleMisinformation
example_text = "Trump says Russia probe will be fair, but timeline unclear: NYT"
print(predict_fake_news(example_text))


Real News


In [28]:
# Test exampleMisinformation
example_text = "Rainbow has 7 colors"
print(predict_fake_news(example_text))

Real News


In [29]:
# Test exampleMisinformation
example_text = "Rainbow has 9 colors"
print(predict_fake_news(example_text))

Fake News
